# Tinder automated like
This app allows you to send Likes on tinder automatically.

facebook_token, which expires almost in an hour, can be got in the url after pusshing OK button in https://www.facebook.com/dialog/oauth?client_id=464891386855067&redirect_uri=fbconnect://success&scope=basic_info%2Cemail%2Cpublic_profile%2Cuser_about_me%2Cuser_activities%2Cuser_birthday%2Cuser_education_history%2Cuser_friends%2Cuser_interests%2Cuser_likes%2Cuser_location%2Cuser_photos%2Cuser_relationship_details&response_type=token&__mref=message

facebook_id can be found from http://findmyfbid.com

In [24]:
import pynder
import datetime
import sqlite3
import time
import os
import json

In [25]:
facebook_id = 'your_id'
facebook_auth_token = 'your_token'

In [26]:
# create a file, tinder_token.json in the same folder as this file with following content. Replace id and token with yours.
# {"facebook_id":"your_id",
# "facebook_auth_token":"your_token"}
token_filename = 'tinder_token.json'
token_path = os.path.exists(token_filename)
if os.path.exists(token_path):
    with open(token_filename) as f:    
        data = json.load(f)
        facebook_id = data['facebook_id']
        facebook_auth_token = data['facebook_auth_token']

In [27]:
session = pynder.Session(facebook_id, facebook_auth_token)

In [28]:
latlon = {'shibuya':(35.6618, 139.7041),
         'todai': (35.7127, 139.7620),
         'ginza': (35.6721, 139.7708)}

In [29]:
session.update_location(*latlon['shibuya']) # updates latitude and longitude for your profile

{'status': 200}

In [30]:
profile = session.profile  # your profile. If you update its attributes they will be updated on Tinder.

Then, add people into database. Sometimes a boy/girl is shown which you already liked. When you see a boy/girl again, skip photos, schools, jobs, common_connections and instagram_username if the same thing is already in database, otherwise insert new row.

In [40]:
conn = sqlite3.connect('tinder.db')
conn.isolation_level = None
c = conn.cursor()

In [32]:
c.execute('''CREATE TABLE IF NOT EXISTS girls
             (id text primary key, name text, bio text, age integer, birth_date timestamp,
             ping_time text, distance_km real, like integer, match integer)''')
c.execute('''CREATE TABLE IF NOT EXISTS photos
             (id text, photo_url text, scraped_date date, unique(id, photo_url))''')
c.execute('''CREATE TABLE IF NOT EXISTS schools
             (id text, school text, scraped_date date, unique(id, school))''')
c.execute('''CREATE TABLE IF NOT EXISTS jobs
             (id text, job text, scraped_date date, unique(id, job))''')
c.execute('''CREATE TABLE IF NOT EXISTS common_connections
             (id text, friend text, scraped_date date, unique(id, friend))''')
c.execute('''CREATE TABLE IF NOT EXISTS instagram_username
             (id text, username text, scraped_date date, unique(id, username))''')

In [33]:
cnt = 0
today = datetime.datetime.today()
while session.can_like_in < 0:
    try:
        users = session.nearby_users() # returns a list of users nearby
    except KeyError as e:
        print(e)
        time.sleep(10)
        continue
    girls = [(user.id, user.name, user.bio, user.age, user.birth_date, datetime.datetime.strptime(user.ping_time, '%Y-%m-%dT%H:%M:%S.%fZ'),
             user.distance_km, 1, 0) for user in users]
    photos = [(user.id, photo, today) for user in users for photo in user.get_photos(width='640') if user.photos] # (id, photo_url)
    schools = [(user.id, school, today) for user in users for school in user.schools] # (id, school)
    jobs = [(user.id, job, today) for user in users for job in user.jobs] # (id, job)
    common_connections = [(user.id, conn, today) for user in users if 'common_friends' in user._data for conn in user.common_connections] # (id, friend)
    instagram_username = [(user.id, user.instagram_username, today) for user in users if user.instagram_username] # (id, username)
    try:
        c.execute('begin')
        c.executemany('INSERT OR REPLACE INTO girls VALUES (?,?,?,?,?,?,?,?,?)', girls)
        c.executemany('INSERT OR IGNORE INTO photos VALUES (?,?,?)', photos)
        c.executemany('INSERT OR IGNORE INTO schools VALUES (?,?,?)', schools)    
        c.executemany('INSERT OR IGNORE INTO jobs VALUES (?,?,?)', jobs)
        c.executemany('INSERT OR IGNORE INTO common_connections VALUES (?,?,?)', common_connections)
        c.executemany('INSERT OR IGNORE INTO instagram_username VALUES (?,?,?)', instagram_username)
        c.execute('end')
    except:
        c.execute('rollback')
    matches = [user.superlike() if ('University of Tokyo' in user.schools or '東京大学' in user.schools) or ('common_friends' in user._data and len(user.common_connections)>8) else user.like() for user in users]
    cnt += len(users)
    time.sleep(0.5)
    print('You liked {} girls.'.format(cnt))
nexttime = datetime.datetime.now() + datetime.timedelta(seconds=session.can_like_in)
print('You can like again at '+nexttime.strftime("%Y-%m-%d %H:%M:%S"))

You liked 11 girls.
You liked 21 girls.
'bio'
'bio'
'bio'
You liked 32 girls.
You liked 43 girls.
You liked 54 girls.
You liked 65 girls.
You liked 76 girls.
'bio'
'bio'
'bio'
'bio'
'bio'
You liked 86 girls.
'bio'
'bio'
'bio'
'bio'
'bio'
'bio'
'bio'
'bio'
'bio'
'bio'
'bio'
'bio'
'bio'
'bio'
'bio'
'bio'
You liked 97 girls.
You liked 107 girls.
'bio'
'bio'
You liked 116 girls.
You liked 127 girls.
You can like again at 2017-03-24 02:46:02


In [34]:
conn.commit()
conn.close()

In [48]:
s = c.execute('select * from schools').fetchall()
for row in s:
    if 'UTokyo' in row[1] or '東京大学' in row[1]:
        print(row)

('58c49ec5d4158bda0bb6123b', '東京大学 | UTokyo', '2017-03-17 15:54:50.291470')
('58d35086aa75cfd90f3b6d9d', 'The University of Tokyo / UTokyo', '2017-03-23 14:40:27.666783')
